In [1]:
import os
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB

In [2]:
def concat_dataframes(path, df_type):
    dfs_list = []
    dfs_rows_len_list = []
    
    for user in os.listdir(path):
        for file in os.listdir(os.path.join(path, user)):
            if file.find(df_type) != -1:
                df = pd.read_csv(os.path.join(path, user, file))
                
                df = df.drop(["timestamp"], axis=1)
#                 df = (df - df.min()) / (df.max() - df.min())
                
                df["user"] = int(user.split('_')[1])
                
                dfs_list.append(df)
    
    return pd.concat(dfs_list, ignore_index=True)


def split_users_into_two_classes(df, valid_user_label, intruder_label):
    df.loc[df["user"] != valid_user_label, "user"] = intruder_label
    df.loc[df["user"] == valid_user_label, "user"] = 1
    return df  


def create_file_for_results(data_type):
    res_folder = '.\\_results'
    if os.path.exists(res_folder) is False:
        os.makedirs(res_folder)
    
    file = os.path.join(res_folder, data_type + '_results.json')
    if os.path.exists(file) is False:
        with open(file, 'w') as f:
            json.dump({'stub': None}, f)
        
    return file    


def update_file_with_results(file_path, results_dict):
    import collections.abc

    def update(d, u):
        for k, v in u.items():
            if isinstance(v, collections.abc.Mapping):
                d[k] = update(d.get(k, {}), v)
            else:
                d[k] = v
        return d
    
    with open(file_path, 'r') as f:
        res = json.load(f)
    
    res = update(res, results_dict)
    
    with open(file_path, 'w') as f:
        json.dump(res, f, sort_keys=True, indent=2)
        
        
def get_dict_with_results(json_path):
    with open(json_path, 'r') as f:
        res = json.load(f)
    return res    


def get_broadcasts_list(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    
    return [x.replace('\n', '') for x in lines]


def fit_label_encoder(broadcasts_list):
    le = LabelEncoder()
    le.fit(broadcasts_list)
    return le


def get_broadcasts_dataframe(data_path, window_type, window_size):
    return concat_dataframes(os.path.join(data_path, window_type, window_size), 'broadcasts'), create_file_for_results('broadcasts')

In [3]:
def get_train_split(df, valid_user, intruder):
    train_df = df[df.user != intruder]
    intruder_label = 0

    train_df = split_users_into_two_classes(train_df.copy(), valid_user, intruder_label)   
    
    valid_user_count = train_df[train_df.user == 1].shape[0]
    intruder_count = train_df[train_df.user == intruder_label].shape[0]

    if valid_user_count < intruder_count:
        train_df = train_df.drop(train_df[train_df.user == intruder_label].sample(intruder_count - valid_user_count).index)
    else:
        train_df = train_df.drop(train_df[train_df.user == 1].sample(valid_user_count - intruder_count).index)
         
    dataset = train_df.to_numpy().copy()
    np.random.shuffle(dataset)

    X = dataset[:, :-1]
    y = dataset[:, -1]    
    
    return X, y


def get_test_split(df):
    df = df.drop('events_count', axis=1)
    df = df.fillna(0)
        
    test_dataset = df.to_numpy().copy()
    
    timestamps = test_dataset[:, 0]
    X_test = test_dataset[:, 1:-1]
    y_test = test_dataset[:, -1]
    
    return X_test, [int(x) for x in y_test], timestamps


def get_test_df(
    data_path,
    events_wnd,
    window_type,
    window_size,
    valid_user,
    intruder,
    data_type,
    index_1,
    index_2
):
    test_df = pd.read_csv(
        os.path.join(
            data_path,
            events_wnd,
            window_type,
            window_size,
            "_".join(["user", str(valid_user)]),
            "_".join([str(data_type), str(index_1), str(index_2), str(intruder)]) + ".data.csv",
        )
    )
    
    return test_df


def test_model(
    results_file,
    model,
    X_test,
    y_test,
    model_tag,
    df_type,
    window_type,
    window_size,
    valid_user,
    intruder,
    index_1,
    index_2,
    timestamps
):
    predict = model.predict(X_test)
    proba = model.predict_proba(X_test)
       
    print("TYPE: ", df_type)
    print('Model = ', model_tag, ', valid user = ', valid_user, ', intruder = ', intruder, ", ", index_1, '_', index_2)    
    print("--------------------------------------------------------------------------------")
        
    results = {
        df_type: {
            "fixed": {
                "fixed": {
                    model_tag: {
                        "valid_user": {
                            str(valid_user): {
                                "intruder": {
                                    str(intruder): {
                                        "valid_sample_index": {
                                            str(index_1): {
                                                "intruder_sample_index": {
                                                    str(index_2): {
                                                        "test": y_test,
                                                        "predict": predict.tolist(),
                                                        "proba": proba.tolist(),
                                                        "timestamps": timestamps.tolist()
                                                    }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }    
    
    update_file_with_results(results_file, results)
    print('Accuracy score = ', accuracy_score(y_test, predict))
    print("--------------------------------------------------------------------------------")
    
    
def simulate(
    users_count,
    samples_count,
    model,
    model_tag,
    data_path,
    test_data_path,
    df_type,
    events_wnd,
    window_type,
    window_size,
    le
):       
    for valid_user in range(1, users_count + 1):
        for intruder in range(1, users_count + 1):
            if valid_user != intruder:
                df, results_file = get_broadcasts_dataframe(data_path, window_type, window_size)
                
                df['action'] = le.transform(df.action)
                
                X_train, y_train = get_train_split(df, valid_user, intruder)

                model.fit(X_train, y_train)

                for valid_user_sample in range(samples_count):
                    for intruder_sample in range(samples_count):
                        
                        test_df = get_test_df(
                            test_data_path,
                            events_wnd,
                            window_type,
                            window_size,
                            valid_user,
                            intruder,
                            df_type,
                            index_1=valid_user_sample,
                            index_2=intruder_sample,
                        )
                        
                        test_df['action'] = le.transform(test_df.action)
                        X_test, y_test, timestamps = get_test_split(test_df)

                        test_model(
                            results_file,
                            model,
                            X_test,
                            y_test,
                            model_tag,
                            df_type,
                            window_type,
                            window_size,
                            valid_user,
                            intruder,
                            index_1=valid_user_sample,
                            index_2=intruder_sample,
                            timestamps=timestamps
                        )

### Simulation settings
### ***

In [4]:
DATA_PATH = '..\\..\\scripts\\_features_all'
TEST_DATA_PATH = '..\\..\\scripts\\_features_events_full'

DATA_TYPE = "broadcasts"
BROADCASTS_LIST_FILE = "..\\learning\\broadcasts.list"

EVENTS_WND = '15min'
WINDOW_TYPE = "rolling"
WINDOW_SIZE = "60s"

SAMPLES_COUNT = 10
USERS_COUNT = 8

broadcasts_list = get_broadcasts_list(BROADCASTS_LIST_FILE)
le = fit_label_encoder(broadcasts_list)

In [5]:
model_params = {
    'min_categories': len(le.classes_)
}

model_tag = "NaiveBayes"
model = CategoricalNB(**model_params)

In [6]:
DATA_TYPES = ["broadcasts"]
WINDOW_SIZES = ["60s"]
WINDOW_TYPES = ["rolling"]

### ***

In [ ]:
for data_type in DATA_TYPES:
    for wnd_type in WINDOW_TYPES:
        for wnd_size in WINDOW_SIZES:
            simulate(
                USERS_COUNT,
                SAMPLES_COUNT,
                model,
                model_tag,
                DATA_PATH,
                TEST_DATA_PATH,
                data_type,
                EVENTS_WND,
                wnd_type,
                wnd_size,
                le
            )

TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  0 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.8275862068965517
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  0 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.6951871657754011
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  0 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.7894736842105263
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  0 _ 3
-------------------------------------------------------------------------------

Accuracy score =  0.6224489795918368
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  3 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.8506224066390041
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  3 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.8015267175572519
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  3 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.8384458077709611
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , va

Accuracy score =  0.5943396226415094
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  6 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.5967741935483871
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  6 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.4519230769230769
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  6 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.5362318840579711
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , va

Accuracy score =  0.6063829787234043
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  9 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.36363636363636365
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  9 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.29333333333333333
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  2 ,  9 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.3
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1

Accuracy score =  0.6666666666666666
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  3 ,  2 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.39615384615384613
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  3 ,  2 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.8428571428571429
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  1 , intruder =  3 ,  2 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.6875
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user = 

...

Accuracy score =  0.5058823529411764
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  2 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.22357723577235772
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  2 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.2708333333333333
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  2 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.22134387351778656
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , 

Accuracy score =  0.425531914893617
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  5 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.5098039215686274
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  5 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.5268817204301075
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  5 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.46551724137931033
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , va

Accuracy score =  0.2508710801393728
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  8 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.43478260869565216
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  8 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.48863636363636365
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  2 ,  8 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.39622641509433965
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes ,

Accuracy score =  0.275
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  3 ,  1 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.6275303643724697
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  3 ,  1 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.5789473684210527
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  3 ,  1 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.6385542168674698
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7

Accuracy score =  0.3794642857142857
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  3 ,  4 _ 0
--------------------------------------------------------------------------------
Accuracy score =  0.631578947368421
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  3 ,  4 _ 1
--------------------------------------------------------------------------------
Accuracy score =  0.5263157894736842
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , valid user =  7 , intruder =  3 ,  4 _ 2
--------------------------------------------------------------------------------
Accuracy score =  0.6666666666666666
--------------------------------------------------------------------------------
TYPE:  broadcasts
Model =  NaiveBayes , val